In [1]:
# 모듈 로딩 --------- 해야함
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

# 저장된 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained('./koelectra_ner_model2')
model = AutoModelForTokenClassification.from_pretrained('./koelectra_ner_model2')

# 새로운 입력 문장에 대해 예측 수행
sentence = "원할머니 보쌈에서 보쌈 중자하나랑 파전하나 주문해줘."
encoding = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True)

with torch.no_grad():
    outputs = model(**encoding)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)

# 예측된 레이블 확인
tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'][0])
predicted_labels = predictions[0].cpu().numpy()

for token, label in zip(tokens, predicted_labels):
    print(f"Token: {token} : Label: {label}")

c:\Users\KDP-23\anaconda3\envs\imc_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OSError: Incorrect path_or_model_id: './koelectra_ner_model2'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [ ]:
# 새로운 입력 문장에 대해 예측 수행
sentence = "원할머니 보쌈에서 보쌈 중자하나, 파전하나 주문해줘."
encoding = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True)

with torch.no_grad():
    outputs = model(**encoding)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)

# 예측된 레이블 확인
tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'][0])
predicted_labels = predictions[0].cpu().numpy()

for token, label in zip(tokens, predicted_labels):
    print(f"Token: {token} : Label: {label}")

Token: [CLS] : Label: 1
Token: 원 : Label: 1
Token: ##할 : Label: 2
Token: ##머니 : Label: 2
Token: 보 : Label: 2
Token: ##쌈 : Label: 2
Token: ##에서 : Label: 0
Token: 보 : Label: 3
Token: ##쌈 : Label: 4
Token: 중 : Label: 4
Token: ##자 : Label: 4
Token: ##하나 : Label: 5
Token: , : Label: 0
Token: 파 : Label: 3
Token: ##전 : Label: 4
Token: ##하나 : Label: 5
Token: 주문 : Label: 0
Token: ##해 : Label: 0
Token: ##줘 : Label: 0
Token: . : Label: 0
Token: [SEP] : Label: 1


In [ ]:
# 새로운 입력 문장에 대해 예측 수행
sentence = "원할머니 보쌈에서 보쌈 중자하나 파전하나 주문해줘."
encoding = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True)

with torch.no_grad():
    outputs = model(**encoding)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)

# 예측된 레이블 확인
tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'][0])
predicted_labels = predictions[0].cpu().numpy()

for token, label in zip(tokens, predicted_labels):
    print(f"Token: {token} : Label: {label}")

Token: [CLS] : Label: 2
Token: 원 : Label: 1
Token: ##할 : Label: 2
Token: ##머니 : Label: 2
Token: 보 : Label: 2
Token: ##쌈 : Label: 2
Token: ##에서 : Label: 0
Token: 보 : Label: 3
Token: ##쌈 : Label: 4
Token: 중 : Label: 4
Token: ##자 : Label: 4
Token: ##하나 : Label: 5
Token: 파 : Label: 4
Token: ##전 : Label: 4
Token: ##하나 : Label: 5
Token: 주문 : Label: 0
Token: ##해 : Label: 0
Token: ##줘 : Label: 0
Token: . : Label: 0
Token: [SEP] : Label: 2


확인 결과 >> 현재 토큰나이저 이상있다는걸 판별

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch
# 기존 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained('./MODEL/koelectra_v3_ner_model2')

# 입력 문장
sentence = "중화루에서 깐풍기 소자 하나랑 나홀로 세트 b번 두개, 야끼우동 세개 주문할게요"

# 기본 토큰화 및 인코딩
encoding = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True)

# input_ids와 attention_mask 그대로 확인
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']

print("기존 encoding:", encoding)
print("기존 input_ids:", input_ids)
print("기존 attention_mask:", attention_mask)

# 기본 토큰화를 한글자 단위로 후처리
tokens = tokenizer.tokenize(sentence)
list_text = []
text_token = []
print("tokens:",tokens)
for token in tokens:
    if token.startswith("##"):  # 특수 기호 제거
        list_text[-1] += token[2:]
    else:
        list_text.append(list(token))  

for a in list_text:
    text_token.extend(a)

# 한글자 단위 결과 출력
print("한글자 단위 토큰:", text_token)

# input_ids를 한글자 단위로 다시 매핑 (필요 시 직접 ID를 생성)
char_input_ids = [tokenizer.convert_tokens_to_ids(t) for t in text_token]
print("한글자 단위 input_ids:", char_input_ids)

In [ ]:
# input_ids를 torch 텐서로 변환
char_input_ids = torch.tensor([char_input_ids])  # 2차원 텐서로 변환
attention_mask = torch.ones_like(char_input_ids)  # 모든 토큰에 대해 마스크 생성

# 저장된 모델 로드
model = AutoModelForTokenClassification.from_pretrained('./MODEL/koelectra_v3_ner_model2')
model.eval()

# 추론
with torch.no_grad():
    outputs = model(input_ids=char_input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)  # 가장 높은 확률을 가진 클래스 선택

In [2]:
# 토큰과 예측된 레이블 연결
# tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
predicted_labels = predictions[0].cpu().numpy()

# 결과 출력
for token, label in zip(text_token, predicted_labels):
    print(f"Token: {token} Label: {label}")

NameError: name 'predictions' is not defined